In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
class apmblog(scrapy.Spider):
    name = 'apm_blog'
    allowed_domains = ['scoutapm.com']
    start_urls = ['https://scoutapm.com/blog/categories/engineering']
    custom_settings = {"FEEDS":{"tmp/apm_blog.csv":{"format":"csv"}}}
    
    def parse(self,response):
        posts = response.xpath("//div[@class='post-partial']")
        for post in posts:
            link = post.css('a').attrib['href']
            url = response.urljoin(link)
            yield scrapy.Request(url = url, callback= self.parse_details)
         #pagination   
        next_page = response.xpath("//a[@class='glyphicon glyphicon-menu-right']").attrib['href']
        if next_page is not None:
            next_p = response.urljoin(next_page)
            yield scrapy.Request(url = next_p,callback=self.parse)
            
    def parse_details(self,response):
        name = response.css('h1::text').extract_first()
        author = response.css('a.author-link::text').extract_first()
        alink = response.css('a.author-link').attrib['href']
        author_link = response.urljoin(alink)
        post_link = response.url
        yield {
            'Name':name,
            'Author':author,
            'Author Profile':author_link,
            'Post': post_link
        }

In [3]:
process = CrawlerProcess()
process.crawl(apmblog)
process.start()

2022-07-14 14:25:15 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-07-14 14:25:15 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.2.0, Python 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 21.0.0 (OpenSSL 1.1.1l  24 Aug 2021), cryptography 3.4.8, Platform Windows-8.1-6.3.9600-SP0
2022-07-14 14:25:15 [scrapy.crawler] INFO: Overridden settings:
{}
2022-07-14 14:25:15 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-07-14 14:25:15 [scrapy.extensions.telnet] INFO: Telnet Password: c2e6c49183914f2b
2022-07-14 14:25:15 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2022-07-14 14:25:16 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloader